Use this notebook as a template to complete the Capstone. This notebook assumes that you have successfully run the previous notebook 05.1_C3_Capstone_Setup_MDF and have at least one MLModel deployed as `LIVE` on the test population segment. It also assumes that you have a trained and tuned pipeline wihch has acheived an F1 score of at least 0.8 that you have used to create an MLModel at the end of the notebook 04_C3_Model_Development_Experimentation. Once you have this MLModel created, you can use this notebook to deploy that model as a `CHALLENGER` model. 

Your goal is to have a `CHALLENGER` model that achieves an F1-Score of at least 0.80 on the Test Segment. Please don't train on the test segment. :)

# Deploy Challenger Models

Necessary imports:

In [ ]:
import pandas as pd
import time
from datetime import timedelta


def monitor_job_status(job, sleepFor=10):
    
    while (job.status().status not in ["completed", "failed", "completed_with_errors"]):
        print(job.status().status)
        time.sleep(sleepFor)
        
    return job.status()

### Retrieve the MLPopulation Segments we currently have

In [ ]:
pd.DataFrame(c3.MLPopulationSegment.fetch().objs.toJson())

### Get Test Segment id:

In [ ]:
# Look in the table above to find the id of the MLPopulation segment for TestingBulbs.
# Make sure to use the id that is attached to an MLProject (does not have NaN in the MLProject column).

test_segment = c3.MLPopulationSegment.get('MLPopulationSegmentIdForTestingBulbs')

In [ ]:
c3.MLPopulationSegment.refreshCalcFields(c3.RefreshCalcFieldsSpec(calcFieldsToRefresh=['primaryConfiguration'],
                                                                  ids=[test_segment.id]))


### What is the current primary deployed model on the Test Segment?

In [ ]:
primary_model_name = test_segment.evaluate(
    c3.EvaluateSpec(filter=f'id=="{test_segment.id}"',
                    projection='primaryConfiguration.predictionModels[0].model.pipeline.steps[0].name'
)).tuples[0].cells[0].value()
print(f"The primary configuration has a model with name: {primary_model_name}.")

In [ ]:
# # see all models deployed in this MLProject and their deployment status

# pd.DataFrame(c3.MLProjectSubjectToModelRelation.fetch().objs.toJson())

# # you can also view this table in the developer console to better view the nested/linked fields with
# # c3Grid(MLProjectSubjectToModelRelation.fetch())

In [ ]:
primary_model_dict = pd.DataFrame(c3.MLProjectSubjectToModelRelation.fetch({
                                                                            "filter": "isPrimary == true"
                                                                          }).objs.toJson())["to"].tolist()
ids = []
for item in primary_model_dict:
    ids.append(item["id"])
    
ids = list(set(ids))
# print(ids)
if len(ids) == 1:
    primary_model_id = ids[0]
    print(f"The primary model deployed on the test segment has the id: {primary_model_id}.")
elif len(ids) > 1: 
    print("There appears to be more than 1 LIVE model deployed in this project!")
    print(ids)
else:
    print("No LIVE model could be found for this project!")

### Retrieve a MLModel that you wish to deploy as `CHALLENGER`:

The id here should be the id of a previously trained and tuned MLModel from **Section 13** of your experimentation notebook:

In [ ]:
challenger_model = c3.MLModel.get('MyTrainedAndTunedMLModelId') # MLModel id from the last cell in notebook 04

### Deploy model as a `CHALLENGER` Model on the Test Segment:

In [ ]:
prediction_config = test_segment.deployModels([challenger_model.configuration], c3.MLModelLabel.CHALLENGER)

In [ ]:
test_segment.updateModels()

### Check that your model has successfully deployed as a `CHALLENGER` Model on the Test Segment:

In [ ]:
# pd.DataFrame(c3.MLProjectSubjectToModelRelation.fetch().objs.toJson())

In [ ]:
# pd.DataFrame(pd.DataFrame(c3.MLProjectSubjectToModelRelation.fetch().objs.toJson())["status"])
pd.DataFrame(c3.MLProjectSubjectToModelRelation.fetch({
                                                                            "filter": f"to == '{challenger_model.id}'"
                                                                          }).objs.toJson())["status"].tolist()[0]

### 🎉🎉🎉
### Awesome! You've just deployed your trained, tuned model in an existing production system as a ```CHALLENGER``` model, using all of the features and hyperparameters you determined, exactly as you designed them.
### 🎉🎉🎉

### Now let's compare the ```CHALLENGER``` model to the ```LIVE``` model.

Run a `Prediction Job` to automatically generate and persist predictions for all active models on the test segment:

In [ ]:
pred_job = test_segment.predict(None, c3.MLPredictionJobOptions(batchSize=1), 
                                      c3.MLModelPredictSpec(allActiveModels=True,
                                      pipelineSpec=c3.MLProcessSpec(disableGpu=True)))


In [ ]:
# pred_job.status()

monitor_job_status(pred_job)

### Compare `LIVE` and `CHALLENGER` models:

Retrieve and plot predictions for one bulb in the test segment:

In [ ]:
predictions_emr = c3.MLModel.evalMetrics(spec=c3.EvalMetricsSpec(
    ids=[challenger_model.id], expressions=["MLProjectPrediction"], 
    start="2016-01-01", end="2021-01-01",
    bindings=[{"subjectId": "SMBLB23"}], # change your subject id here to another id in the test segment to see other predictions
    interval="HOUR",
    resultKey=c3.Lambda.fromPython("lambda expression, bindings: expression + ('_' + bindings.get('featName', '') if expression == 'MLProjectContribution' else '')")
))

In [ ]:
c3.EvalMetricsResult.toPandas(predictions_emr, multiIndexed=True).droplevel(0).plot(figsize=(16, 4))

Run a `Scoring Job` to automatically generate and persist predictions for all active models on the test segment:

In [ ]:
score_job = test_segment.score(None, c3.MLScoreJobOptions(batchSize=10), 
                                      c3.MLModelScoreSpec(allActiveModels=True))


In [ ]:
while (score_job.status().status not in ['completed', 'completed with errors', 'failed']):
    print(score_job.status().status)
    time.sleep(10)
    
# score_job.status()

monitor_job_status(score_job)

Retrieve Results from the Scoring Job for the `CHALLENGER` model:

In [ ]:
from datetime import timedelta
start = challenger_model.get("scores.data.this").scores[0].data[0].start
scores_emr = c3.MLModel.evalMetrics(spec=c3.EvalMetricsSpec(
                ids=[challenger_model.id], expressions=["Score"], 
                start=start - timedelta(1), end=start + timedelta(hours=12),
                bindings=[{'scoringMetricName': 'MLF1ScoreMetric'}],
                interval="DAY",
                resultKey=c3.Lambda.fromPython("lambda expression, bindings: expression + '_' + bindings['scoringMetricName']")
))


In [ ]:
scores_df = c3.EvalMetricsResult.toPandas(scores_emr, multiIndexed=True).droplevel(0)

In [ ]:
scores_df

In [ ]:
c3.EvalMetricsResult.toPandas(scores_emr, multiIndexed=True).droplevel(0).plot(figsize=(12, 4), marker='x', grid=True, subplots=True)

Call an instance of the `LIVE` model deployed on the test segment, using the id of the primary model you retrieved earlier in this notebook.

In [ ]:
live_model = c3.MLModel.get(primary_model_id)

Retrieve Results from the Scoring Job for the `LIVE` model:

In [ ]:
start = live_model.get("scores.data.this").scores[0].data[0].start
scores_emr = c3.MLModel.evalMetrics(spec=c3.EvalMetricsSpec(
                ids=[live_model.id], expressions=["Score"], 
                start=start - timedelta(1), end=start + timedelta(hours=12),
                bindings=[{'scoringMetricName': 'MLF1ScoreMetric'}],
                interval="DAY",
                resultKey=c3.Lambda.fromPython("lambda expression, bindings: expression + '_' + bindings['scoringMetricName']")
))



In [ ]:
scores_df = c3.EvalMetricsResult.toPandas(scores_emr, multiIndexed=True).droplevel(0)

In [ ]:
scores_df

In [ ]:
c3.EvalMetricsResult.toPandas(scores_emr, multiIndexed=True).droplevel(0).plot(figsize=(12, 4), marker='x', grid=True, subplots=True)

## **Congratulations** 
on successfully deploying multiple models on the test segment and successfully comparing their performance! If you are satisfied with the performance of your `CHALLENGER` model, submit this model's id for your capstone submission. If you would like to go back to notebook 04 and continue to iterate and experiment to improve model performance, feel free to do so -- you can deploy as many `CHALLENGER` models as you would like!